In [3]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from scipy.interpolate import UnivariateSpline
import os

In [7]:
# ERA5
home_path = os.path.expanduser("~")

path = '/DataFiles'
path2 = '/Library/Mobile Documents/com~apple~CloudDocs/Documents/Documents - MacBook Air/Python/Ice Cores/data/model/ccsm4_last_millenium'

allEra = nc.Dataset(home_path + path2 + '/era5total.nc')

In [24]:
print(allEra.variables['tp'])

<class 'netCDF4.Variable'>
int16 tp(time, expver, latitude, longitude)
    scale_factor: 4.09993986774206e-07
    add_offset: 0.013434486031357766
    _FillValue: -32767
    missing_value: -32767
    units: m
    long_name: Total precipitation
unlimited dimensions: 
current shape = (547, 2, 301, 3600)
filling on


In [23]:
#ERA5 Data processing
tasBase = np.array(allEra.variables['t2m'][:252, 0])
prBase = np.array(allEra.variables['tp'][:252, 0])

eraTemps = np.zeros([20, 301, 3600])
eraAccums = np.zeros([20, 301, 3600])

for i in range(20):
    eraTemps[i] = np.mean(tasBase[(i*12):(i*12+12)], axis=0)
    eraAccums[i] = np.mean(prBase[(i*12):(i*12+12)], axis=0) * 365000

eraAccums[eraAccums < 0] = np.nan

# Regridding to the resolution of CCSM4
rolledTemps = np.roll(eraTemps, 1800, axis=2)
rolledAccums = np.roll(eraAccums, 1800, axis=2)

tempRegrid = np.zeros([20, 32, 3600])
secondRegrid = np.zeros([20, 32, 7200])
latInd = np.array(pd.read_csv(home_path + path2 + '/latInd.csv'))

firstTemps = np.zeros([20, 32, 3600])
firstAccums = np.zeros([20, 32, 3600])

secondTemps = np.zeros([20, 32, 7200])
secondAccums = np.zeros([20, 32, 7200])

i = 0
for item in latInd:
    firstTemps[:, i, :] = rolledTemps[:, item[0]]
    firstAccums[:, i, :] = rolledAccums[:, item[0]]
    i = i + 1

for j in range(20):
    for i in range(32):
        secondTemps[j, i, :7198] = np.dstack((firstTemps[j, i, :-1], firstTemps[j, i, :-1] + np.diff(firstTemps[j, i]) / 2.0)).ravel()
        secondAccums[j, i, :7198] = np.dstack((firstAccums[j, i, :-1], firstAccums[j, i, :-1] + np.diff(firstAccums[j, i]) / 2.0)).ravel()

finalTemps = secondTemps[:, :, ::25]
finalAccums = secondAccums[:, :, ::25]



In [ ]:
basins = np.loadtxt(home_path + path2 + '/ant_full_drainagesystem_polygons.txt')

        
basinAvgs = np.zeros(27)
b = 1
tempSum = 0
count = 0


basNo = 27
yearSums = np.zeros(45)
yearCount = np.zeros(45)
rSums = np.zeros(28)
ceSums = np.zeros(28)
cellCount = 0.0

areas = [0, 501448, 830000, 1565727, 247879, 190978, 610797, 493593, 159742, 146029, 919041, 257286, 722224, 1109771, 710953, 125183, 265243, 1852382, 269776, 382018, 201853, 217404, 214497, 91266, 160848, 34621, 43085, 54146]
basinMaster = np.zeros([27, 45])

for k in range(28):
    if k != 0:
        for i in basins:
            if i[2] == k:
                lat = i[0]
                lon = i[1]

                if lon < 0:
                    lon = lon + 360

                latInd = round(lat / 2) + 45
                lonInd = round(lon / 2)

                if lonInd == 180:
                    lonInd = 0


                if finalTemps[0, int(latInd), int(lonInd)] > 0.0:
                    for j in range(45):
                        yearCount[j] = yearCount[j] + 1
                        yearSums[j] = yearSums[j] + finalAccums[j, int(latInd), int(lonInd)]

        timeSeries = np.divide(yearSums, yearCount)
        for i in range(45):
            basinMaster[k-1, i] = 0.9167*(timeSeries[i]*areas[k-1])/1000000
        print(k)
        yearSums = np.zeros(45)
        yearCount = np.zeros(45)
        b = 1
        tempSum = 0
        count = 0
        basinAvgs = np.zeros(27)

AP = np.sum(basinMaster[23:], axis=0)
WAIS = np.sum(basinMaster[17:23], axis=0) + basinMaster[0]
EAIS = np.sum(basinMaster[1:17], axis=0) 
AIS = AP + WAIS + EAIS

for i in range(45):
    print(AIS[i])

np.savetxt("ERA5AccumMetric_AIS.csv", AIS)
np.savetxt("ERA5AccumMetric_WAIS.csv", WAIS)
np.savetxt("ERA5AccumMetric_EAIS.csv", EAIS)

avgAIS = np.zeros([1, 45])
avgWAIS = np.zeros([1, 45])
avgEAIS = np.zeros([1, 45])
print("________________________")
for i in range(27):
    avgAIS[0, :] = avgAIS[0, :] + areas[i] * basinMaster[i]
    if i == 0 or (i > 16 and i < 23):
        avgWAIS[0, :] = avgWAIS[0, :] + areas[i] * basinMaster[i]
    elif i > 0 and i < 17:
        avgEAIS[0, :] = avgEAIS[0, :] + areas[i] * basinMaster[i]
    
avgAIS = avgAIS / np.sum(areas, axis=0)
avgWAIS = avgWAIS / (np.sum(areas[17:23], axis=0) + areas[0])
avgEAIS = avgEAIS / np.sum(areas[1:17], axis=0)
    
#AP = np.sum(basinMaster[23:], axis=0)
#WAIS = np.sum(basinMaster[17:23], axis=0) + basinMaster[0]
#EAIS = np.sum(basinMaster[1:17], axis=0) 
#AIS = AP + WAIS + EAIS

print(avgAIS)

#np.savetxt("ERA5TempMetric_AIS.csv", avgAIS)
#np.savetxt("ERA5TempMetric_WAIS.csv", avgWAIS)
#np.savetxt("ERA5TempMetric_EAIS.csv", avgEAIS)